In [1]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

%pylab inline

import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import nibabel as nib
from nibabel.viewers import OrthoSlicer3D

train_path = glob.glob('./脑PET图像分析和疾病预测挑战赛公开数据/Train/*/*')
test_path = glob.glob('./脑PET图像分析和疾病预测挑战赛公开数据/Test/*')

np.random.shuffle(train_path)
np.random.shuffle(test_path)

In [3]:
for path in train_path:
    img = nib.load(path)
    print(path, img.dataobj[:, :, :, 0].shape)

./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/19.nii (128, 128, 47)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/21.nii (128, 128, 47)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/NC/4.nii (128, 128, 47)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/8.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/2.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/NC/10.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/NC/11.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/NC/13.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/6.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/18.nii (128, 128, 47)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/NC/2.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/22.nii (128, 128, 47)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/NC/6.nii (128, 128, 47)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/23.nii (128, 128, 47)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/13.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/14.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/17.nii (128, 128, 47)
./脑PET图像分析和疾病预测挑战赛公开数据/Train/MCI/1.nii (128, 128, 63)
./脑PET图像分析和疾病预测挑战赛公开数据/

In [4]:
DATA_CACHE = {}
class XunFeiDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        if self.img_path[index] in DATA_CACHE:
            img = DATA_CACHE[self.img_path[index]]
        else:
            img = nib.load(self.img_path[index]) 
            img = img.dataobj[:,:,:, 0]
            DATA_CACHE[self.img_path[index]] = img
            
        idx = np.random.choice(range(img.shape[-1]), 50)
        # idx.sort()
        img = img[:, :, idx]
        img = img.astype(np.float32)
        
        # img /= 255.0
        # img -= 1
        
        # print(img.shape)
        if self.transform is not None:
            img = self.transform(image = img)['image']
        
        img = img.transpose([2,0,1])
        return img,torch.from_numpy(np.array(int('NC' in self.img_path[index])))
    
    def __len__(self):
        return len(self.img_path)

In [5]:
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
                
        model = models.resnet18(True)
        model.conv1 = torch.nn.Conv2d(50, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 2)
        self.resnet = model
        
    def forward(self, img):        
        out = self.resnet(img)
        return out

In [6]:
def train(train_loader, model, criterion, optimizer):
    model.train()
    train_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 20 == 0:
            print(loss.item())
            
        train_loss += loss.item()
    
    return train_loss/len(train_loader)
            
def validate(val_loader, model, criterion):
    model.eval()
    
    val_acc = 0.0
    
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)
            
            val_acc += (output.argmax(1) == target).sum().item()
            
    return val_acc / len(val_loader.dataset)

def predict(test_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    
    test_pred = []
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(test_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            test_pred.append(output.data.cpu().numpy())
            
    return np.vstack(test_pred)

In [7]:
import albumentations as A

train_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[:-10],
            A.Compose([
            A.RandomRotate90(),
            A.RandomCrop(120, 120),
            A.HorizontalFlip(p=0.5),
            A.RandomContrast(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
        ])
    ), batch_size=2, shuffle=True, num_workers=1, pin_memory=False
)

val_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[-10:],
            A.Compose([
            A.RandomCrop(120, 120),
            # A.HorizontalFlip(p=0.5),
            # A.RandomContrast(p=0.5),
        ])
    ), batch_size=2, shuffle=False, num_workers=1, pin_memory=False
)

test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(test_path,
            A.Compose([
            A.RandomCrop(128, 128),
            A.HorizontalFlip(p=0.5),
            A.RandomContrast(p=0.5),
        ])
    ), batch_size=2, shuffle=False, num_workers=1, pin_memory=False
)


/home/lyz/.local/lib/python3.9/site-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [8]:
model = XunFeiNet()
model = model.to('cuda')
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.AdamW(model.parameters(), 0.001)

/home/lyz/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  warnings.warn(
/home/lyz/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
for _  in range(1):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc  = validate(val_loader, model, criterion)
    train_acc = validate(train_loader, model, criterion)
    
    print(train_loss, train_acc, val_acc)

0.6160869598388672
1.3067537385970354 0.55 0.1


In [10]:
pred = None

for _ in range(10):
    if pred is None:
        pred = predict(test_loader, model, criterion)
    else:
        pred += predict(test_loader, model, criterion)

In [11]:
submit = pd.DataFrame(
    {
        'uuid': [int(x.split('/')[-1][:-4]) for x in test_path],
        'label': pred.argmax(1)
})

In [12]:
submit['label'] = submit['label'].map({1:'NC', 0: 'MCI'})
submit = submit.sort_values(by='uuid')
submit.to_csv('submit2.csv', index=None)

In [13]:
submit

,uuid,label
25,1,MCI
81,2,NC
45,3,MCI
2,4,NC
52,5,NC
...,...,...
59,96,NC
97,97,NC
66,98,NC
10,99,NC
